base64

In [29]:
import base64
import os
import json

def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    return encoded_string

def convert_images_to_individual_jsons(folder_path):
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(folder_path, filename)
            base64_data = encode_image_to_base64(file_path)
            image_json = {
                "image": base64_data
            }

            # Crear el nombre del archivo JSON
            json_filename = os.path.splitext(filename)[0] + '.json'
            json_filepath = os.path.join(folder_path, json_filename)

            # Guardar
            with open(json_filepath, 'w') as json_file:
                json.dump(image_json, json_file, indent=4)


carpetas = ['imagenes_extraidas_instruction_icono', 'imagenes_extraidas_instruction_maquina']

# Aplicar a cada carpeta
for folder in carpetas:
    convert_images_to_individual_jsons(folder)


In [ ]:
import os
import json
import base64
from PIL import Image
import io

carpetas = [
    'imagenes_extraidas_instruction_maquina',
    'imagenes_extraidas_instruction_icono',
]

def determinar_tipo(nombre_carpeta):
    if "maquina" in nombre_carpeta:
        return "machine"
    elif "icono" in nombre_carpeta:
        return "icon"
    return "unknown"

for carpeta in carpetas:
    tipo = determinar_tipo(carpeta)

    for filename in os.listdir(carpeta):
        if filename.endswith('.json'):
            file_path = os.path.join(carpeta, filename)

            with open(file_path, 'r') as file:
                json_data = json.load(file)
                base64_data = json_data["image"]

            try:
                print(f"Procesando {filename}, longitud de base64: {len(base64_data)}")

                image_data = base64.b64decode(base64_data)
                image = Image.open(io.BytesIO(image_data))

                size = f"{image.size[0]}x{image.size[1]}"
                format = image.format

                updated_data = {
                    "pag": int(filename.split('_')[1]),
                    "img": int(filename.split('_')[3].split('.')[0]),
                    "type": tipo,
                    "size": size,
                    "format": format,
                    "save_image": base64_data
                }

                with open(file_path, 'w') as file:
                    json.dump(updated_data, file, indent=4)

            except Exception as e:
                print(f"Error al procesar el archivo {filename}: {e}")
